In [1]:
import pandas as pd
import numpy as np
import tempfile
import os.path
import calendar

# Ouverture du dataset avec remplacements des données manquantes afin qu'elles soient detectable par pandas en tant que Na/NaN

In [2]:
missing_values = ["n/a", "na", "--", "-"]
#file_path = "D:\_IMSD\Data viz'\dataset.csv"
file_path = "/home/sam/Documents/cours/DataViz/project_dataviz2_imsd-master/dataset.csv"
dataset = pd.read_csv(file_path, encoding="ISO-8859-1", na_values = missing_values)

/home/sam/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# copy of DataFrame => cleaned one
df = dataset.copy()
df.head()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [4]:
pd.set_option('display.max_columns', None)
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181691 entries, 0 to 181690
Columns: 135 entries, eventid to related
dtypes: float64(55), int64(22), object(58)
memory usage: 187.1+ MB


(181691, 135)

# Suppression des attaques n'ayant pas de date complète

#### Suppr
ession des lignes n'ayant pas de mois ou de jours (~ 0.50%)

In [5]:
print((df[df['imonth'] == 0].shape[0] + df[df['iday'] == 0].shape[0]) / df.shape[0] * 100.)
#print((891/181691)*100)

0.5014007298105024


In [6]:
df.drop(df[(df.imonth == 0) | (df.iday ==0)].index, inplace=True)

In [7]:
#df.info()
df.shape[0]

180800

Suppresion des villes 'unknown' ou nan

In [8]:
#df[df.city == 'unknown'].size
df.drop(df[(df.city == 'unknown') | df['city'].isnull() == 1].index, inplace=True)
#df[df['city'].isnull() == 1] #18964

# Remplacer les numéros de mois & jours par leur abbréviation

In [9]:
df['imonth'] = df['imonth'].apply(lambda x: calendar.month_abbr[x])

# Renomer les colonnes


In [10]:
df.rename(columns={'iyear':'year','imonth':'month','iday':'day','country':'country_code', 'country_txt':'country','region_txt':'region',
                     'crit1':'criteria','attacktype1_txt':'attacktype','targtype1_txt':'targtype',
                     'targsubtype1_txt':'targsubtype','corp1':'targeted_entity','natlty1_txt':'targeted_nationality',
                     'gname':'perpetrator_group','gsubname':'perpetrator_details','claimmode_txt':'claim_mode',
                     'compclaim':'competing_claim','weaptype1_txt':'weapon_type','weapsubtype1_txt':'weapon_subtype',
                     'weapdetail':'weapon_details','nwound':'nb_vitcims_injured','nwoundte':'nb_perpetrators_injured',
                     'propextent_txt':'property_damage','ransomamt':'amount_ransom','hostkidoutcome_txt':'hostkidoutcome'
                    },inplace=True)

In [11]:
df.head()


,eventid,year,month,day,approxdate,extended,resolution,country_code,country,region,region,provstate,city,latitude,longitude,specificity,vicinity,location,summary,criteria,crit2,crit3,doubtterr,alternative,alternative_txt,multiple,success,suicide,attacktype1,attacktype,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype,targsubtype1,targsubtype,targeted_entity,target1,natlty1,targeted_nationality,targtype2,targtype2_txt,targsubtype2,targsubtype2_txt,corp2,target2,natlty2,natlty2_txt,targtype3,targtype3_txt,targsubtype3,targsubtype3_txt,corp3,target3,natlty3,natlty3_txt,perpetrator_group,perpetrator_details,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claim_mode,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,competing_claim,weaptype1,weapon_type,weapsubtype1,weapon_subtype,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4,weapsubtype4_txt,weapon_details,nkill,nkillus,nkillter,nb_vitcims_injured,nwoundus,nb_perpetrators_injured,property,propextent,property_damage,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,amount_ransom,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome,hostkidoutcome,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,Jul,2,NaN,0,NaN,58,Dominican Republic,2,Central America & Caribbean,NaN,Santo Domingo,18.456792,-69.951164,1.0,0,NaN,NaN,1,1,1,0.0,NaN,NaN,0.0,1,0,1,Assassination,NaN,NaN,NaN,NaN,14,Private Citizens & Property,68.0,Named Civilian,NaN,Julio Guzman,58.0,Dominican Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MANO-D,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
5,197001010002,1970,Jan,1,NaN,0,NaN,217,United States,1,North America,Illinois,Cairo,37.005105,-89.176269,1.0,0,NaN,1/1/1970: Unknown African American assailants ...,1,1,1,0.0,NaN,NaN,0.0,1,0,2,Armed Assault,NaN,NaN,NaN,NaN,3,Police,22.0,"Police Building (headquarters, station, school)",Cairo Police Department,Cairo Police Headquarters,217.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Black Nationalists,NaN,NaN,NaN,NaN,NaN,To protest the Cairo Illinois Police Deparment,0.0,NaN,NaN,0,-99.0,-99.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,Firearms,5.0,Unknown Gun Type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Several gunshots were fired.,0.0,0.0,0.0,0.0,0.0,0.0,1,3.0,Minor (likely < $1 million),NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"The Cairo Chief of Police, William Petersen, r...","""Police Chief Quits,"" Washington Post, January...","""Cairo Police Chief Quits; Decries Local 'Mili...","Christopher Hewitt, ""Political Violence and Te...",Hewitt Project,-9,-9,0,-9,NaN
6,197001020001,1970,Jan,2,NaN,0,NaN,218,Uruguay,3,South America,Montevideo,Montevideo,-34.891151,-56.187214,1.0,0,NaN,NaN,1,1,1,0.0,NaN,NaN,0.0,0,0,1,Assassination,NaN,NaN,NaN,NaN,3,Police,25.0,Police Security Forces/Officers,Uruguayan Police,Juan Maria de Lucah/Chief of Directorate of in...,218.0,Uruguay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupamaros (Uruguay),NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,Firearms,2.0,Automatic or Semi-Automatic Rifle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Automatic firearm,0.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
7,197001020002,1970,Jan,2,NaN,0,NaN,217,United States,1,North America,California,Oakland,37.

# Création du dataset pour le random button: conserver uniquement les lignes n'ayant aucune valeur manquante des événements à compter 1997

In [12]:
#higher_1997 = df.copy()

In [13]:
#data = higher_1997[higher_1997['iyear'] >= 1997]
#data.head()
#data.info()
#data.head()

In [14]:
#random_button = data[
#    (data.iyear.notnull()) & (data.imonth.notnull()) & (data.iday.notnull()) & (data.imonth.notnull()) &
#    (data.country_txt.notnull()) & (data.city.notnull()) & (data.summary.notnull()) & (data.attacktype1_txt.notnull()) &
#    (data.targsubtype1_txt.notnull()) & (data.gname.notnull()) & (data.nkill.notnull())]

In [15]:
#random_button.info()

# Renomer les colonnes

# Suppression des colonnes non utilisées pour l'analyse

##### Pour cette analyse, nous nous baserons uniquement sur valeurs principales => Nous conserverons les données "1" de crit, attacktype_txt, targtype_txt, corp, ,target, targtype ...

In [16]:
df.drop([
    'region','specificity','vicinity','doubtterr','alternative','alternative_txt', 'attacktype1',
    'attacktype2','attacktype3', 'targtype1', 'targtype2', 'targtype3', 'targsubtype1', 'targsubtype2', 'targsubtype3',
    'natlty2', 'natlty3', 'claimmode', 'claimmode2', 'claimmode3', 'weaptype1', 'weaptype1', 'weaptype2', 'weaptype3',
    'weaptype4', 'weapsubtype1', 'weapsubtype2', 'weapsubtype3', 'weapsubtype4', 'nkillus', 'nwoundus', 'nhostkidus',
    'ransomamtus', 'hostkidoutcome', 'INT_LOG', 'INT_IDEO', 'INT_MISC', 'INT_ANY', 'related', 'crit2', 'crit3',
    'natlty1', 'attacktype2_txt', 'targtype3_txt', 'guncertain3', 'claim2', 'claimmode2_txt', 'claim3', 'claimmode3_txt',
    'attacktype2_txt','attacktype3_txt', 'targtype2_txt', 'targsubtype2_txt', 'corp2', 'target2', 'natlty2_txt', 
    'targsubtype3_txt', 'corp3','target3', 'natlty3_txt', 'gname2', 'gsubname2', 'gname3', 'gsubname3', 'guncertain2',
    'weaptype2_txt', 'weapsubtype2_txt', 'weaptype3_txt', 'weapsubtype3_txt', 'weaptype4_txt', 'weapsubtype4_txt',
    'guncertain1','propextent','ransompaidus', 'dbsource','approxdate','extended','resolution','property','property_damage',
    'propvalue','propcomment','nhours','ndays','divert','kidhijcountry','ransom','amount_ransom','ransompaid','ransomnote',
    'hostkidoutcome','nreleased'   
], axis=1, inplace=True)


In [17]:
df.shape

(180325, 44)

# Standardisation des pays

In [30]:
#df[df['city']== 'Belgrade']
# Obj : verifier si une ville a deux country differente :

#df.city.unique().size #36550
#uniqueCountry = df.groupby('city')['country'].nunique()
#uniqueCountry[uniqueCountry > 1].size #971
#uniqueCountry[uniqueCountry > 1]
    
#print(df.groupby('city')['country'].transform('nunique'))
#df[['city', 'country']].groupby(['city']).count()
df[df['city']=='Erlangen']

#df[df['country'].str.contains(" Germany ")]['city'].unique()

,eventid,year,month,day,country_code,country,provstate,city,latitude,longitude,location,summary,criteria,multiple,success,suicide,attacktype,targtype,targsubtype,targeted_entity,target1,targeted_nationality,perpetrator_group,perpetrator_details,motive,individual,nperps,nperpcap,claimed,claim_mode,competing_claim,weapon_type,weapon_subtype,weapon_details,nkill,nkillter,nb_vitcims_injured,nb_perpetrators_injured,ishostkid,nhostkid,addnotes,scite1,scite2,scite3
2451,197406190007,1974,Jun,19,362,Germany,Bavaria,Erlangen,49.599937,11.0063,NaN,NaN,1,0.0,1,0,Facility/Infrastructure Attack,Police,"Police Building (headquarters, station, school)",NaN,Police Station,Germany,Unknown,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Incendiary,NaN,Incendiary,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
12516,198012190004,1980,Dec,19,362,Germany,Bavaria,Erlangen,49.599937,11.0063,NaN,NaN,1,0.0,1,0,Assassination,Journalists & Media,Newspaper Journalist/Staff/Facility,Jewish newspaper,"Publsiher Shomo Lewin + Frida Poesahke, companion",Germany,Unknown,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Firearms,Handgun,Pistol,2.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [19]:
#for i in uniqueCountry.index:        
#    if(uniqueCountry[i] > 1):
#        print(df[df['city']==i][['city','country']])

In [20]:
new_country = {
    'Czech Republic': ['Prague', 'Usti Nad Labem', 'Halickuv Brod', 'Hradec Kralove'],
    'Slovak Republic': ['Bratislava'],
    'Serbia': ['Velki Trnovac','Medvedja','Muhovac','Pozarevac','Oraovica','Cerevajka','Bujic','Banjica',
                'Bujanovac','Lucane','Belgrade', 'Bujanovac', 'Vrbas', 'Lucani', 'Bujanovac', 'Dobrosin',
                'Suva Reka', 'Podujeva','Donje Prekaze','Subotica','Batlava','Velika Reka','Loznica','Novi Sad',
                'Cacak','Zemun','Valjevo','Nis','Surcin','Veliki Trnovac','Konculj','Borovac','Vranje'],
    'Montenegro': ['Borovica','Podgorica', 'Berane', 'Podgorica', 'Kolasin','Berane','Cetinje','Podgorica','Budva'],
    'Macedonia': ['Skopje','Careva Cesma','Prilep'],
    'Slovenia': ['Ljubljana','Stari Trg'],
    'Croatia': ['Kijevo','Dubrovnik','Osijek','Benkovac','Zagreb','Pribudic','Dvor na Uni','Knin','Polaca','Sotin',
                 'Petrinja','Titova Korenica','Pakrac','Lisani','Dolac'],
    'Kosovo': ['Streoc','Dolovo','Pristina', 'Glogovac','Titova Mitrovica','Stimlje','Decani','Peje','Kosovska Mitrovica',
                'Krpimej','Donje Ljupce','Surkis','Istinic','Bogicevica','Srbica','Runik','Prizren','Kline','Gornja Klina',
                'Djakovica','Klincina','Grabanicë','Malisevo','Opterusa','Magurë','Orlat','Grabc','Ljubenic','Blace','Voksh',
                'Dulje','Tusuz','Donji Crnobreg','Komoran','Podujevë','Prekale','Kosovo Polje','Urosevac','Besinje','Kruska'],
    'Bosnia-Herzegovina': ['Sarajevo','Odzak','Banja Luka','Mostar','Kalesija','Bosanski Brod','Bijelina','Celopek'],
    'Albania': ['Vucitrn'],
    'Russia': ['Moscow','Buynaksk','Kara-Su','Novgorod','Vasyurinskaya','Sverdlovsk','Grebenskaya','Temirgoye',
              'Irkutsk'],
    'Georgia': ['Sukhumi','Meghri district','Tbilisi'],
    'Armenia': ['Yerevan','Aygepar','Kirovakan'],
    'Azerbaijan': ['Yevlakh','Nagorno-Karabakh district','Khankendi','Baganis-Airum','Chaily','Tartar district',
                  'Stepanakert','Gyandzha','Kazakh','Kazakh district','Agstafa district','Aterk'],
    'Kyrgyzstan': ['Osh'],
    'Ukraine': ['Feodosia'],
    'Estonia': ['Tallinn','Ikla'],
    'Latvia': ['Riga','Ainazi'],
    'Lithuania': ['Lavoriskes','Birzai','Salociai','Vilnius','Medininkai'],
    'Eritrea': ['Asmara'],
    'Germany':['Berlin', 'Munich', 'Frankfurt', 'West Berlin', 'Stuttgart',
       'Wiesloch', 'Heidelberg', 'Kladow', 'Hamburg', 'Unknown',
       'Karlsruhe', 'Bonn', 'Kronberg im Taunus', 'Dusseldorf',
       'Kaiserslautern', 'Bochum', 'Bad Kreuznach', 'Kampen',
       'Monchengladbach', 'Boblingen', 'Hanover', 'Essen', 'Nuremberg',
       'Darmstadt', 'Cologne', 'Erlangen', 'Mannheim', 'Gottingen',
       'Bremen', 'Bamberg', 'Ludwigshafen', 'Mainz',
       'Ludwigshafen am Rhein', 'Fulda', 'Kronach', 'Dortmund', 'Singen',
       'Neuss', 'Hamm', 'Moers', 'Wiesbaden', 'Offenbach',
       'Schemmerhofen', 'Meisau', 'Freiburg', 'Friedrichshofen',
       'Pforzheim', 'Bielefeld', 'Hanau', 'Bad Hersfeld', 'Aachen',
       'Giessen', 'Memmelsdorf', 'Bogen', 'Flensburg', 'Radolfzell',
       'Zweibrucken', 'Westerburg', 'Rheindahlen', 'Verden', 'Nottuln',
       'Zeppelinheim', 'Krefeld', 'Muenster', 'Osnabruck', 'Kassel',
       'Waldshut-Tiengen', 'Lichterfelde', 'Ramstein-Miesenbach',
       'Zirndorf', 'Herford', 'Untergruppenbach', 'Bad Toelz',
       'Gelnhausen', 'Butzbach', 'Ginsheim-Gustavsburg',
       'Schwabisch Gmund', 'Magdeburg', 'Aalen', 'Oberammergau',
       'Reutlingen', 'Wertheim', 'Hohenhaus', 'Geesthacht',
       'Bad Harzburg', 'Gauting', 'Groningen', 'Iggingen', 'Koblenz',
       'Badbergen', 'Freisen', 'Nohfelden', 'Bad Cannstatt', 'Limburg',
       'Baunatal', 'Baal', 'Neustadt an der Weinstrasse', 'Wuppertal',
       'Bad Godesberg', 'Feucht', 'Heidenburg', 'Kirchheimbolanden',
       'Immenstaad', 'Marburg', 'Teveren', 'Hagen', 'Detmold district',
       'Leverkusen', 'Tutzing', 'Tubingen', 'Lahr', 'Hedemunden', 'Wesel',
       'Brunswick', 'Goppingen', 'Bayreuth', 'Duisburg', 'Gross-Gerau',
       'Gremmendorf', 'Unna', 'Wildenrath', 'Bad Homburg', 'Monheim',
       'Eschborn', 'Hameln']
    
}

In [21]:
# Nombre de villes à renomer 
count = 0 
for i in new_country:
    if isinstance(new_country[i],list):
        count += len(new_country[i])

print(count)
for countries, cities in new_country.items():
    for city in cities:
        print('Il y a en ' + countries + ' toutes les villes suivantes ' + city)

280
Il y a en Czech Republic toutes les villes suivantes Prague
Il y a en Czech Republic toutes les villes suivantes Usti Nad Labem
Il y a en Czech Republic toutes les villes suivantes Halickuv Brod
Il y a en Czech Republic toutes les villes suivantes Hradec Kralove
Il y a en Slovak Republic toutes les villes suivantes Bratislava
Il y a en Serbia toutes les villes suivantes Velki Trnovac
Il y a en Serbia toutes les villes suivantes Medvedja
Il y a en Serbia toutes les villes suivantes Muhovac
Il y a en Serbia toutes les villes suivantes Pozarevac
Il y a en Serbia toutes les villes suivantes Oraovica
Il y a en Serbia toutes les villes suivantes Cerevajka
Il y a en Serbia toutes les villes suivantes Bujic
Il y a en Serbia toutes les villes suivantes Banjica
Il y a en Serbia toutes les villes suivantes Bujanovac
Il y a en Serbia toutes les villes suivantes Lucane
Il y a en Serbia toutes les villes suivantes Belgrade
Il y a en Serbia toutes les villes suivantes Bujanovac
Il y a en Serbia t

In [22]:
for i in new_country:
    #print(i)
    #print(new_country[i])
    #df[df['city'] == 'A']
    index_to_update = df[df['city'].isin(new_country[i])].index
    #print(index_to_update)
    df.at[index_to_update,'country'] = i 
df

Int64Index([ 37130,  37706,  38079,  38414,  42785,  47722,  53153,  54358,
             56934,  60596,  60652,  63108,  63767,  64360,  67075,  67300,
             67339,  87379,  92701, 132828, 140833, 140834, 149099, 150047,
            150197, 158632, 166357],
           dtype='int64')
Int64Index([ 54508,  58999,  59761,  62252,  63191,  63206,  65016,  65191,
             66086,  67837,  69556,  69891, 166738],
           dtype='int64')
Int64Index([  1970,  10781,  13700,  18257,  37774,  38123,  49636,  52760,
             54422,  58766,  60737,  60769,  62155,  62243,  62662,  63033,
             63986,  64381,  64544,  64874,  65149,  65539,  65550,  66071,
             66072,  66879,  66960,  67041,  68435,  68642,  68903,  69464,
             69564,  69643,  69902,  70011,  70038,  70179,  70277,  70322,
             70372,  70520,  70589,  71170,  71208,  71656,  71777,  71778,
             71787,  71798,  71837,  71845,  71853,  71879,  71898,  71904,
             71925,  7

,eventid,year,month,day,country_code,country,provstate,city,latitude,longitude,location,summary,criteria,multiple,success,suicide,attacktype,targtype,targsubtype,targeted_entity,target1,targeted_nationality,perpetrator_group,perpetrator_details,motive,individual,nperps,nperpcap,claimed,claim_mode,competing_claim,weapon_type,weapon_subtype,weapon_details,nkill,nkillter,nb_vitcims_injured,nb_perpetrators_injured,ishostkid,nhostkid,addnotes,scite1,scite2,scite3
0,197000000001,1970,Jul,2,58,Dominican Republic,NaN,Santo Domingo,18.456792,-69.951164,NaN,NaN,1,0.0,1,0,Assassination,Private Citizens & Property,Named Civilian,NaN,Julio Guzman,Dominican Republic,MANO-D,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,1.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
5,197001010002,1970,Jan,1,217,United States,Illinois,Cairo,37.005105,-89.176269,NaN,1/1/1970: Unknown African American assailants ...,1,0.0,1,0,Armed Assault,Police,"Police Building (headquarters, station, school)",Cairo Police Department,Cairo Police Headquarters,United States,Black Nationalists,NaN,To protest the Cairo Illinois Police Deparment,0,-99.0,-99.0,0.0,NaN,NaN,Firearms,Unknown Gun Type,Several gunshots were fired.,0.0,0.0,0.0,0.0,0.0,NaN,"The Cairo Chief of Police, William Petersen, r...","""Police Chief Quits,"" Washington Post, January...","""Cairo Police Chief Quits; Decries Local 'Mili...","Christopher Hewitt, ""Political Violence and Te..."
6,197001020001,1970,Jan,2,218,Uruguay,Montevideo,Montevideo,-34.891151,-56.187214,NaN,NaN,1,0.0,0,0,Assassination,Police,Police Security Forces/Officers,Uruguayan Police,Juan Maria de Lucah/Chief of Directorate of in...,Uruguay,Tupamaros (Uruguay),NaN,NaN,0,3.0,NaN,NaN,NaN,NaN,Firearms,Automatic or Semi-Automatic Rifle,Automatic firearm,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
7,197001020002,1970,Jan,2,217,United States,California,Oakland,37.791927,-122.225906,Edes Substation,1/2/1970: Unknown perpetrators detonated explo...,1,0.0,1,0,Bombing/Explosion,Utilities,Electricity,Pacific Gas & Electric Company,Edes Substation,United States,Unknown,NaN,NaN,0,-99.0,-99.0,0.0,NaN,NaN,Explosives,Unknown Explosive Type,NaN,0.0,0.0,0.0,0.0,0.0,NaN,"Damages were estimated to be between $20,000-$...",Committee on Government Operations United Stat...,"Christopher Hewitt, ""Political Violence and Te...",NaN
8,197001020003,1970,Jan,2,217,United States,Wisconsin,Madison,43.076592,-89.412488,NaN,"1/2/1970: Karl Armstrong, a member of the New ...",1,0.0,1,0,Facility/Infrastructure Attack,Military,Military Recruiting Station/Academy,R.O.T.C.,"R.O.T.C. offices at University of Wisconsin, M...",United States,New Year's Gang,NaN,To protest the War in Vietnam and the draft,0,1.0,1.0,1.0,Letter,NaN,Incendiary,Molotov Cocktail/Petrol Bomb,Firebomb consisting of gasoline,0.0,0.0,0.0,0.0,0.0,NaN,The New Years Gang issue a communiqué to a loc...,"Tom Bates, ""Rads: The 1970 Bombing of the Army...","David Newman, Sandra Sutherland, and Jon Stewa...","The Wisconsin Cartographers' Guild, ""Wisconsin..."
9,197001030001,1970,Jan,3,217,United States,Wisconsin,Madison,43.072950,-89.386694,NaN,"1/3/1970: Karl Armstrong, a member of the New ...",1,0.0,1,0,Facility/Infrastructure Attack,Government (General),Government Building/Facility/Office,Selective Service,Selective Service Headquarters in Madison Wisc...,United States,New Year's Gang,NaN,To protest the War in Vietnam and the draft,0,1.0,1.0,0.0,NaN,NaN,Incendiary,Gasoline or Alcohol,Poured gasoline on the floor and lit it with a...,0.0,0.0,0.0,0.0,0.0,NaN,"Karl Armstrong's girlfriend, Lynn Schultz, dro...",Committee on Government Operations United Stat...,"Tom Bates, ""Rads: The 1970 Bombing of the Army...","David Newman, Sandra Sutherland, and Jon Stewa..."
10,197001050001,1970,Jan,1,217,United States,Wisconsin,Baraboo,43.468500,-89.744299,NaN,NaN,1,0.0,0,0,Bombing/Explosion,Military,Military Barracks/Base/Headquarters/Checkpost,NaN,Badger Army ammo depot.,United States,"Weather Underground, Weathermen",NaN,NaN,0,NaN,N

In [31]:
df[df['city']=='Stepanakert']


,eventid,year,month,day,country_code,country,provstate,city,latitude,longitude,location,summary,criteria,multiple,success,suicide,attacktype,targtype,targsubtype,targeted_entity,target1,targeted_nationality,perpetrator_group,perpetrator_details,motive,individual,nperps,nperpcap,claimed,claim_mode,competing_claim,weapon_type,weapon_subtype,weapon_details,nkill,nkillter,nb_vitcims_injured,nb_perpetrators_injured,ishostkid,nhostkid,addnotes,scite1,scite2,scite3
43302,199007150014,1990,Jul,15,359,Azerbaijan,Azerbaijan,Stepanakert,39.829805,46.759851,NaN,NaN,1,0.0,1,0,Assassination,Private Citizens & Property,House/Apartment/Residence,Stepanakert Apt.,"Residence, Manager Yevgeniya Ishjanova",Soviet Union,Unknown,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Explosives,Unknown Explosive Type,Explosive,1.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
44259,199010140012,1990,Oct,14,359,Azerbaijan,Azerbaijan,Stepanakert,39.829805,46.759851,NaN,NaN,1,0.0,0,0,Bombing/Explosion,Private Citizens & Property,Political Party Member/Rally,Azerbaijani Organizing Commitee,Railway car used as HQ,Soviet Union,Unknown,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Explosives,Unknown Explosive Type,Explosive,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
46407,199105100011,1991,May,10,359,Azerbaijan,Azerbaijan,Stepanakert,39.829805,46.759851,NaN,NaN,1,0.0,1,0,Bombing/Explosion,Government (General),Government Building/Facility/Office,government,"offices, district Commandant and Azerbaijan Co...",Soviet Union,Unknown,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Explosives,"Projectile (rockets, mortars, RPGs, etc.)",Rifle propelled grenade,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [24]:
'''for i in test['country']:
    if i == 'Yugoslavia':
        for countries, cities in new_country.items():
            for city in cities:
                test['country'] = countries   '''     

"for i in test['country']:\n    if i == 'Yugoslavia':\n        for countries, cities in new_country.items():\n            for city in cities:\n                test['country'] = countries   "

In [25]:
'''for i in test.index:
    if test['country'] == 'West Germany (FRG)':
        test.at[i, 'ifor'] = 'Germany'
    elif test['country'] == 'East Germany (GDR)':
        test.at[i, 'ifor'] = 'Germany'''
        

"for i in test.index:\n    if test['country'] == 'West Germany (FRG)':\n        test.at[i, 'ifor'] = 'Germany'\n    elif test['country'] == 'East Germany (GDR)':\n        test.at[i, 'ifor'] = 'Germany"

In [26]:
'''test[test['city'] == 'Berlin']'''

"test[test['city'] == 'Berlin']"

In [27]:
'''test[test['city'] == 'Bonn']'''

"test[test['city'] == 'Bonn']"

In [28]:
"""
soviet_union = test[test['country'] == 'Soviet Union']
soviet_union['city'].unique()

czechoslovakia = test[test['country'] == 'Czechoslovakia']
czechoslovakia['city'].unique()

yugoslavia = test[test['country'] == 'Yugoslavia']
yugoslavia['city'].unique()

df[df['perpetrator_group'] == 'Unknown']

drop = df.copy()

drop.dropna(axis=0, how='any',inplace=True)
random_button.to_csv('terrorism_random_button.csv')
"""


"\nsoviet_union = test[test['country'] == 'Soviet Union']\nsoviet_union['city'].unique()\n\nczechoslovakia = test[test['country'] == 'Czechoslovakia']\nczechoslovakia['city'].unique()\n\nyugoslavia = test[test['country'] == 'Yugoslavia']\nyugoslavia['city'].unique()\n\ndf[df['perpetrator_group'] == 'Unknown']\n\ndrop = df.copy()\n\ndrop.dropna(axis=0, how='any',inplace=True)\nrandom_button.to_csv('terrorism_random_button.csv')\n"

In [32]:
df.to_csv('terrorism_global.csv', sep=',', index=False, header=False, encoding='utf-8')